# Cross Validation
> Os conjuntos de holdout são um ótimo começo para a validação do modelo. No entanto, usar um único conjunto de treinamento e teste geralmente não é suficiente. A validação cruzada é considerada o padrão ouro quando se trata de validar o desempenho do modelo e é quase sempre usada ao ajustar os hiperparâmetros do modelo. Este capítulo se concentra na execução da validação cruzada para validar o desempenho do modelo. Este é o resumo da palestra "Validação de modelo em Python", via datacamp.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8, 8)

## Os problemas com conjuntos de retenção

### Duas amostras
Após construir vários modelos de classificação com base no tic_tac_toe conjunto de dados, você percebe que alguns modelos não generalizam tão bem quanto outros. Você criou divisões de treinamento e teste exatamente como lhe foi ensinado, então está curioso para saber por que seu processo de validação não está funcionando.

Depois de tentar um treinamento diferente, divisão de teste, você notou diferentes precisões para seu modelo de machine learning. Antes de ficar muito frustrado com os resultados variados, você decidiu ver o que mais poderia estar acontecendo.

In [2]:
tic_tac_toe = pd.read_csv("tic-tac-toe.csv")
tic_tac_toe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [ ]:
# Crie duas amostras diferentes de 200 observações
sample1 = tic_tac_toe.sample(n=200, random_state=1111)
sample2 = tic_tac_toe.sample(n=200, random_state=1171)

# Imprima o número de observações comuns
print(len([index for index in sample1.index if index in sample2.index]))

# Imprima o número de observações na coluna class para ambas as amostras
print(sample1['class'].value_counts())
print(sample2['class'].value_counts())

40
class
True     134
False     66
Name: count, dtype: int64
class
True     123
False     77
Name: count, dtype: int64


Observe que há um número variável de observações positivas para ambos os conjuntos de teste de amostra. Às vezes, criar uma única amostra de retenção de teste não é suficiente para atingir os altos níveis de validação de modelo que você deseja. Você precisa usar algo mais robusto.

### scikit-learn's KFold()
Você acabou de executar o código de um colega que cria um modelo de floresta aleatória e calcula uma precisão fora da amostra. Você notou que o código do seu colega não tinha um estado aleatório, e os erros que você encontrou eram completamente diferentes dos erros que seu colega relatou.

Para obter uma estimativa melhor de quão preciso esse modelo de floresta aleatória será em novos dados, você decidiu gerar alguns índices para usar na cross-validation do KFold.

In [7]:
candy = pd.read_csv("candy-data.csv")
candy.head()

,competitorname,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent
0,100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725
1,3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936
2,One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086
3,One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505
4,Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465


In [8]:
X = candy.drop(['competitorname', 'winpercent'], axis=1).to_numpy()
y = candy['winpercent'].to_numpy()

In [9]:
from sklearn.model_selection import KFold

In [10]:
# Usar KFold
kf = KFold(n_splits=5, shuffle=True, random_state=1111)

# Criar divisões
splits = kf.split(X)

# Imprimir o número de indices
for train_index, val_index in splits:
    print("Número de indices de treinamento: %s" % len(train_index))
    print("Número de indices de validação: %s" % len(val_index))

Número de indices de treinamento: 68
Número de indices de validação: 17
Número de indices de treinamento: 68
Número de indices de validação: 17
Número de indices de treinamento: 68
Número de indices de validação: 17
Número de indices de treinamento: 68
Número de indices de validação: 17
Número de indices de treinamento: 68
Número de indices de validação: 17


Este conjunto de dados tem 85 linhas. Você criou cinco divisões - cada uma contendo 68 índices de treinamento e 17 de validação. Você pode usar esses índices para concluir a validação cruzada de 5 dobras.

### Usando índices KFold
Você já criou `splits`, que contém índices para o conjunto de dados candy-data para concluir a validação cruzada de 5 vezes. Para obter uma estimativa melhor de quão bem o modelo de floresta aleatória de um colega executará em novos dados, você deseja executar esse modelo nos cinco índices diferentes de treinamento e validação que você acabou de criar.

Neste exercício, você usará esses índices para verificar a precisão deste modelo usando as cinco divisões diferentes. Um loop for foi fornecido para auxiliar neste processo.

In [11]:
# Criar divisões
splits = kf.split(X)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
rfc = RandomForestRegressor(n_estimators=25, random_state=1111)

# Acessa os indices de treinamento e validação dos splits
for train_index, val_index in splits:
    # Configura os dados de treinamento e validação
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]

    # Ajusta o modelo de floresta aleatória
    rfc.fit(X_train, y_train)

    # Fazer previsões e imprimir
    predictions = rfc.predict(X_val)
    print("Split accuracy: " + str(mean_squared_error(y_val, predictions)))

Split accuracy: 150.99298148707666
Split accuracy: 171.22206240542593
Split accuracy: 131.72569156195593
Split accuracy: 80.61940183841385
Split accuracy: 221.63020627476214


`KFold()`é um ótimo método para acessar índices individuais ao concluir validação cruzada. Uma desvantagem é precisar de um loop for para trabalhar com os índices.

## cross_val_score() do sklearn

### métodos do scikit-learn
Você decidiu construir um modelo de regressão para prever o número de novos funcionários que sua empresa contratará com sucesso no mês que vem. Você abre um novo script Python para começar, mas rapidamente percebe que o sklearn tem muitos módulos diferentes. Vamos garantir que você entenda os nomes dos módulos, os métodos e qual módulo contém qual método.

Siga as instruções abaixo para carregar todos os métodos necessários para concluir a validação cruzada usando sklearn. Você usará módulos:

- metrics
- model_selection
- ensemble

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer

### Implementar cross_val_score()
Sua empresa criou vários doces novos para vender, mas não tem certeza se deve lançar todos os cinco. Para prever a popularidade desses novos doces, você foi solicitado a construir um modelo de regressão usando o conjunto de dados de doces. Lembre-se de que o valor da resposta é uma porcentagem de vitória frente a frente contra outros doces.

Antes de começar a testar diferentes modelos de regressão, você decidiu executar uma validação cruzada em um modelo de floresta aleatória simples para obter um erro de base para comparar com quaisquer resultados futuros.

In [ ]:
rfc = RandomForestRegressor(n_estimators=25, random_state=1111)
mse = make_scorer(mean_squared_error)

cv = cross_val_score(estimator=rfc, X=X_train, y=y_train, cv=10, scoring=mse)

print(cv.mean())

131.30323056938693


Agora você tem uma pontuação de base para construir. Se decidir construir modelos adicionais ou tentar novas técnicas, você deve tentar obter um erro menor que 155,56. Erros menores indicam que suas previsões de popularidade estão melhorando.

### Leave-one-out-cross-validation
Vamos supor que seu doce favorito não esteja no conjunto de dados de doces e que você esteja interessado na popularidade desse doce. Usar a validação cruzada de 5 dobras treinará em apenas 80% dos dados por vez. O conjunto de dados de doces tem apenas 85 linhas, e deixar de fora 20% dos dados pode atrapalhar nosso modelo. No entanto, usar a validação cruzada de deixar um de fora nos permite aproveitar ao máximo nosso conjunto de dados limitado e lhe dará a melhor estimativa para a popularidade do seu doce favorito!

In [16]:
from sklearn.metrics import mean_absolute_error

mae_score = make_scorer(mean_absolute_error)

scores = cross_val_score(rfc, X, y, cv=85, scoring=mae_score)

print("A média dos erros é: %s." % np.mean(scores))
print("O desvio padrão dos erros é: %s." % np.std(scores))

A média dos erros é: 9.37664643137255.
O desvio padrão dos erros é: 7.416832878420677.
